In [24]:
import numpy as np
import pandas as pd
import os

def rrkm_rate():
    # This script calculates RRKM rates for unimolecular reactions. It requires Density of states for the reactant and sum of states for 
    # transition states, which have been pre-calculated using output of Density Functional Theory calculations and the script Densum which 
    # is part of the Multiwell software destribution (https://multiwell.engin.umich.edu/).
    # The files can be found in the folder /PAH_mol_proc/data/NAME_REACTION/ as reactant.out and ts.out.
    # The script stores the RRKM rate as NAME_REACTION_E0.txt in the folder /PAH_MOL_proc/data/rate/output
    

    # USER INPUT !!!!
    ###########################################
    energy_ev = 0.48 # Energy of the reaction barrier in eV
    
    # Path to input files containing Density of States (DOF) and Sum of States (SOF) for diffent reactions
    # Adjust the path accordingly
    reactant_file = '~/PAH_mol_proc/data/back_to_cora/reactant.out'
    ts_file = '~/PAH_mol_proc/data/back_to_cora/ts.out'
    rrkm_file = f'~/PAH_mol_proc/data/rate_output/back_to_cora_{energy_ev}.txt'
    #############################################
    
    rrkm_file = os.path.expanduser(rrkm_file)


    
    # Constants
    h = 4.135667516e-15 * 8065.6  # Planck's constant in eV*s
    energy_cm = 8065.6   # Conversion eV to cm^-1


    # Reading the input files (assumed format: columns separated by whitespace)
    df_react = pd.read_csv(reactant_file, delim_whitespace=True, header=0, names=['No_react', 'EnMode_react', 'DOS_react', 'SOS_react'])
    df_ts = pd.read_csv(ts_file, delim_whitespace=True, header=0, names=['No_ts', 'EnMode_ts', 'DOS_ts', 'SOS_ts'])

    # Energy calculations
    E0 = energy_ev * energy_cm  # Energy of the barrier in cm^-1
    index_E0 = int(round(E0 / 100))  # Step used for DOS/SOS calculations

    #print(df_react['EnMode_react'][index_E0], E0)

    # Initialize K_E array
    K_E = np.zeros(len(df_react['EnMode_react']) - index_E0)

    # Convert energy to eV
    En_react_eV = df_react['EnMode_react'] / energy_cm

    # Calculate RRKM rate
    for i in range(index_E0, len(df_react['EnMode_react'])):
        K_E[i - index_E0] = df_ts['SOS_ts'][i - index_E0] / df_react['DOS_react'][i] / h


    # Create directory for newfile
    NEWDIR = os.path.dirname(rrkm_file)
    os.makedirs(NEWDIR, exist_ok=True)

        # Writing to output file
    with open(rrkm_file, 'w') as f:
        f.write('# RRKM rate for corannulene isomer \n')
        f.write('En(eV)     K(E) s-1\n')
        for i in range(len(K_E)):
            f.write(f'{En_react_eV[index_E0 + i]:.6f}  {K_E[i]:.6e}\n')
        f.write('        \n')

# Run the function
rrkm_rate()
